<a href="https://colab.research.google.com/github/dealbujav/CE888/blob/master/Lab_5/my_recommender(lab5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Latent Factor Models

In [0]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv", delimiter = ",", header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.00,99.00,99.00,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,99.00,99.00,99.00,99.00,99.00,4.90,99.00,99.00,99.00,99.00,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,99.00,99.00,99.00,-6.46,-1.65,-6.80,-6.41,-6.99,7.23,6.75,-6.99,6.55,99.00,99.00,99.00,99.00,0.49,-0.53,-6.94,-0.49,99.00,6.46,-0.53,99.00,99.00,-7.86,-0.34,99.00,-6.94,99.00,...,0.49,-0.24,99.00,99.00,-3.11,-6.65,99.00,-0.58,6.31,99.00,99.00,-7.86,99.00,99.00,

In [0]:
#Droping the firt column (it only shows the number of score jokes)
df = df.drop(0,1)

###Helper function

In [0]:
arr = df.values
print(arr.shape)

(24983, 100)


In [0]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1810455 (1810455,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [0]:
arr[idx[0][0], idx[1][0]]

7.18

In [0]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
new_arr

array([[-7.82,  8.79, -9.66, ..., 99.  , 99.  , 99.  ],
       [ 4.08, -0.29,  6.36, ..., 99.  , 99.  ,  1.07],
       [99.  , 99.  , 99.  , ..., 99.  , 99.  , 99.  ],
       ...,
       [99.  , 99.  , 99.  , ..., 99.  , 99.  , 99.  ],
       [99.  , 99.  , 99.  , ..., 99.  , 99.  , 99.  ],
       [ 2.43,  2.67, -3.98, ..., 99.  , 99.  , 99.  ]])

##Recommender system based on latent features

In [0]:
n_latent_factors = 2

user_ratings = df.values
user_ratings
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
latent_item_features

array([[0.71192473, 0.79303534],
       [0.41833054, 0.30808855],
       [0.49959424, 0.93464942],
       [0.46871839, 0.42372297],
       [0.98740786, 0.93487315],
       [0.36531644, 0.05786519],
       [0.81758026, 0.54880252],
       [0.2353407 , 0.70130905],
       [0.72237929, 0.39604497],
       [0.89556085, 0.97307892],
       [0.01746839, 0.79638924],
       [0.6580827 , 0.67617088],
       [0.52636192, 0.63576007],
       [0.55048055, 0.32400352],
       [0.04566743, 0.98733557],
       [0.30582045, 0.85455949],
       [0.86716971, 0.68725734],
       [0.77382558, 0.72346378],
       [0.96676461, 0.57929783],
       [0.91641173, 0.71076992],
       [0.23429169, 0.16895719],
       [0.88613526, 0.79784307],
       [0.59159313, 0.76330652],
       [0.60759574, 0.92841019],
       [0.9696531 , 0.82730776],
       [0.89825467, 0.38758159],
       [0.159633  , 0.62884703],
       [0.46289548, 0.8916209 ],
       [0.42901328, 0.07367634],
       [0.04161382, 0.19283123],
       [0.

In [0]:
latent_user_preferences

array([[0.54098845, 0.46483532],
       [0.70976359, 0.65002882],
       [0.81408264, 0.55626794],
       ...,
       [0.17036022, 0.18726303],
       [0.3176437 , 0.61540089],
       [0.23926135, 0.13348658]])

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=30000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 1) == 0:
            print(mse)

In [0]:
sgd(30)

26.148217590720837
25.176373927107775
24.88322795640501
24.644846661065667
24.40905659452659
24.170444019180238
23.929320843991288
23.687088755294553
23.445356416994326
23.205640650869285
22.96919169020713
22.736850333921264
22.508914819985492
22.285012136243353
22.063975323663794
21.843736558163087
21.621260694412456
21.392568769042434
21.152935229411554
20.89737594885363
20.621545921497244
20.32307958366616
20.00317355090336
19.667850913128138
19.328073371606703
18.99807138348405
18.69213707863784
18.421186106827317
18.19068152092959
18.000644395810948


In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[-2.65522938e+00, -3.35493825e+00, -3.94053050e+00, ...,
        -1.33005007e+00, -3.37363167e+00, -1.15067073e+00],
       [ 2.57024074e+00,  2.80399950e+00,  2.92915789e+00, ...,
         2.37035129e+00,  2.52438256e+00,  2.59786297e+00],
       [ 4.75881788e+00,  4.71764746e+00,  4.47739413e+00, ...,
         5.54586698e+00,  3.88180329e+00,  6.39576990e+00],
       ...,
       [-1.51756810e-01, -2.25332667e-01, -2.92245205e-01, ...,
         5.97477190e-03, -2.48943051e-01,  4.66010006e-02],
       [ 7.08920597e-01,  6.31461283e-01,  5.24644285e-01, ...,
         1.00029989e+00,  4.59071753e-01,  1.19141712e+00],
       [ 2.69140519e+00,  2.62056591e+00,  2.43733277e+00, ...,
         3.25262845e+00,  2.11592577e+00,  3.77630955e+00]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [0]:
comparison_data
# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, -2.655229384136751)","(8.79, -3.3549382495993902)","(-9.66, -3.9405305000685993)","(-8.16, -6.600907972207138)","(-7.52, -2.1442163203408358)","(-8.5, -1.4987753007436277)","(-9.85, -3.782577611173663)","(4.17, -3.1642424044814885)","(-8.98, -6.067519739052296)","(-4.76, -2.492078573458998)","(-8.5, -1.3003297242496377)","(-6.75, -1.7313279603087677)","(-7.18, -5.120891134346291)","(8.45, -1.7223084465140246)","(-7.18, -4.873502765628165)","(-7.52, -7.44409351464857)","(-7.43, -3.6249574179312147)","(-9.81, -4.147842484168759)","(-9.85, -3.024464796601891)","(-9.85, -4.774291502570678)","(-9.37, -0.11554751043288085)","(1.5, -3.322104489951516)","(-4.37, -4.608035642624471)","(-9.81, -7.416663293205552)","(-8.5, -3.817351034338349)","(1.12, -1.7193523462288243)","(7.82, 2.113185936092708)","(2.86, -1.6883819442130321)","(9.13, 2.1379647174857523)","(-7.43, -5.664699204990881)","(2.14, 0.21892920035952415)","(-4.08, 1.928014369037405)","(-9.08, -6.765588194002364)","(7.82, -2.4913038639907805)","(5.05, 1.660734442544617)","(4.95, 2.4169417943497895)","(-9.17, -6.455773876533421)","(-8.4, -2.708971344271479)","(-8.4, -2.223790225872602)","(-8.4, -2.8996138187621443)",...,"(8.59, 0.3634834067159359)","(3.59, 1.3426693125810159)","(-6.84, -4.202254272199968)","(-9.03, -5.89975872068971)","(2.82, 0.5706217443109727)","(-1.36, 0.5351772601265281)","(-9.08, -6.486535116495564)","(8.3, 0.9794535031395423)","(5.68, 0.7789329272901303)","(-4.81, -3.750778962828612)","(99.0, -3.4669458416990526)","(99.0, 1.1905659855196993)","(99.0, -1.0810652176691833)","(99.0, -4.681704857177889)","(99.0, -2.9134018403220607)","(99.0, 0.9591452654706054)","(99.0, -1.5584107863745464)","(-9.42, -0.3449403140249608)","(99.0, -2.271241447646093)","(99.0, -1.1207906069893856)","(99.0, -0.18360579002982708)","(-7.72, -1.8802590299622897)","(99.0, 0.3215053840290139)","(99.0, -2.288153815752967)","(99.0, -1.5677870974291728)","(99.0, -3.1139415892025166)","(99.0, -0.19233346572334487)","(99.0, -0.2056273188216909)","(99.0, 2.4281838622427006)","(99.0, -2.487990615443324)","(2.82, -0.40219317549237216)","(99.0, -1.3103030594546692)","(99.0, 0.6530141246404096)","(99.0, -2.2009542108390794)","(99.0, -1.8480302629318301)","(99.0, -1.196954952590482)","(-5.63, -0.7136775219029136)","(99.0, -1.3300500678772922)","(99.0, -3.3736316700057087)","(99.0, -1.15067072945552)"
1,"(4.08, 2.57024074235365)","(-0.29, 2.803999503256261)","(6.36, 2.92915788911584)","(4.37, 3.1193707498917074)","(-2.38, 2.0806562346184267)","(-9.66, 3.014740169946756)","(-0.73, 2.126330003668822)","(-5.34, 1.383820599923548)","(8.88, 3.2231588739144756)","(9.22, 2.692752012157037)","(6.75, 2.91156303053825)","(8.64, 3.1725290354196476)","(4.42, 1.6283656484655227)","(7.43, 3.1660577799854663)","(4.56, 1.3288730205696584)","(-0.97, 1.5873335689858594)","(4.66, 1.5811952992046763)","(-0.68, 1.9490203671339448)","(3.3, 2.3176447909463898)","(-1.21, 2.461552838903453)","(0.87, 2.8250088029494784)","(8.64, 3.247038031594227)","(8.35, 3.1800001432502163)","(9.17, 3.07627025171641)","(0.05, 3.2315804631824117)","(7.57, 3.082759590258699)","(4.71, 2.2433807286252763)","(0.87, 2.990517312415449)","(-0.39, 2.0839720607029815)","(6.99, 3.4514943231146256)","(6.5, 2.6612432447851866)","(-0.92, 2.298558507530106)","(7.14, 3.2023955369111565)","(9.03, 3.0662529998805126)","(-1.8, 2.3917262967609734)","(0.73, 2.0464014647238886)","(7.09, 2.790643570364782)","(3.4, 3.0443065945019896)","(-0.87, 3.122234806243788)","(7.91, 3.1714960680763173)",...,"(-6.7, 2.7730269687233653)","(-3.35, 2.5018777410148627)","(-9.03, 3.187429690644071)","(4.47, 3.025022687042059)","(4.08, 2.5694021048870295)","(-3.83, 2.619282681846858)","(8.74, 3.2329648010928143)","(1.12, 2.4389781524

In [0]:
#MSE
error=(predictions[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mse = sum(error)/len(error)
mse

17.84839279885388

###Changing some hyper-parameters
* learning rate to 0.0005
* number of iterations to 40
* number of latent factors to 3

In [0]:
n_latent_factors2 = 3

user_ratings = df.values
user_ratings
# Initialise as random values
latent_user_preferences2 = np.random.random((user_ratings.shape[0], n_latent_factors2))
latent_item_features2 = np.random.random((user_ratings.shape[1], n_latent_factors2))

In [0]:
def predict_rating2(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference2 = latent_user_preferences2[user_id]
    item_preference2 = latent_item_features2[item_id]
    return user_preference2.dot(item_preference2)


def train2(user_id, item_id, rating, alpha=0.0005):
    
    #print item_id
    prediction_rating2 = predict_rating2(user_id, item_id)
    err =  prediction_rating2 - rating
    #print err
    user_pref_values2 = latent_user_preferences2[user_id][:]
    latent_user_preferences2[user_id] -= alpha * err * latent_item_features2[item_id]
    latent_item_features2[item_id] -= alpha * err * user_pref_values2
    return err
    

def sgd2(iterations=30000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences2.shape[0]):
            for item_id in range(latent_item_features2.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                    err = train2(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 1) == 0:
            print(mse)

In [0]:
sgd2(40)

24.98153316389187
24.03697439480783
23.17762322934597
22.2222536011715
20.80680313805229
19.00536622370653
17.960350787114216
17.606278384955143
17.456070091197166
17.35645509918867
17.26017997816364
17.14634509064829
17.012332256075187
16.873822634702382
16.752982382034528
16.662423909152917
16.601922544460948
16.565039840788124
16.544827897927863
16.53589831006197
16.534564520494577
16.538440450700865
16.546020856482382
16.556368920068522
16.568909107858765
16.583296307785044
16.599334731361427
16.616928040797397
16.63604897073043
16.656721324747227
16.67901007934565
16.703017023347527
16.728880347488623
16.756777162321725
16.786928238639
16.81960444865639
16.85513452294001
16.89391389595234
16.936414648321435
16.983196909042757


In [0]:
predictions2 = latent_user_preferences2.dot(latent_item_features2.T)
predictions2

array([[-2.84716768, -3.64194139, -4.96577298, ..., -2.5215958 ,
        -5.88515438, -2.54481552],
       [ 2.32126604,  4.6498051 ,  4.51685542, ...,  4.31111169,
         2.90185987,  4.97458416],
       [ 6.45468428,  5.91694293,  6.66578642, ...,  6.58677611,
         7.83923072,  6.87650684],
       ...,
       [-0.80107412,  1.2600502 ,  0.13901883, ...,  1.44114328,
        -2.37481962,  2.05188764],
       [-1.53621446, -0.4850533 , -1.8690322 , ...,  0.22063015,
        -4.02303749,  0.66116196],
       [ 3.71214993,  4.09143751,  3.96542184, ...,  4.77320259,
         3.60748723,  5.22118249]])

In [0]:
values = [zip(user_ratings[i], predictions2[i]) for i in range(predictions2.shape[0])]
comparison_data2 = pd.DataFrame(values)
comparison_data2.columns = df.columns

In [0]:
comparison_data2

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, -2.8471676805193202)","(8.79, -3.641941388327117)","(-9.66, -4.965772976004079)","(-8.16, -8.678871105836237)","(-7.52, -2.7280116991601795)","(-8.5, -1.482445505284293)","(-9.85, -4.909319849269626)","(4.17, -4.359036347108692)","(-8.98, -8.014319752128975)","(-4.76, -3.8322790251142775)","(-8.5, -1.733397198232977)","(-6.75, -1.234244911365746)","(-7.18, -6.760847360332822)","(8.45, -1.1575039928959945)","(-7.18, -7.347198873905401)","(-7.52, -10.222830471525901)","(-7.43, -4.755750567870627)","(-9.81, -5.722027022387698)","(-9.85, -4.029150726482165)","(-9.85, -6.878332864163345)","(-9.37, 0.45932079156777983)","(1.5, -4.066290114845169)","(-4.37, -5.9085091936627645)","(-9.81, -10.341814959848174)","(-8.5, -4.42405200511885)","(1.12, -1.528073420201279)","(7.82, 3.2543925901379844)","(2.86, -1.6073991834127086)","(9.13, 3.3214090017526945)","(-7.43, -7.605920241330203)","(2.14, 1.1111708812460617)","(-4.08, 3.205187181355795)","(-9.08, -9.383744684748132)","(7.82, -2.5580369591161474)","(5.05, 2.7235062375568564)","(4.95, 3.538762500322101)","(-9.17, -9.26635047106958)","(-8.4, -3.929164079087716)","(-8.4, -2.2981243407787515)","(-8.4, -3.7341261723110812)",...,"(8.59, 0.8996608279184324)","(3.59, 1.8847069105368102)","(-6.84, -5.964955607336603)","(-9.03, -8.544411547929247)","(2.82, 1.3231564602456392)","(-1.36, 1.3910058797860076)","(-9.08, -9.080227533375924)","(8.3, 1.5425061493309211)","(5.68, 1.6559806714515533)","(-4.81, -4.598131587760924)","(99.0, -6.7636179793650575)","(99.0, 1.6508187822600673)","(99.0, -2.560174594389419)","(99.0, -8.421941075349766)","(99.0, -6.038591494544963)","(99.0, 0.36072415135053415)","(99.0, -4.364195562975526)","(-9.42, -2.5479879638324503)","(99.0, -5.957289485754002)","(99.0, -3.3815735340320887)","(99.0, -1.027648854880718)","(-7.72, -4.588143672453655)","(99.0, -0.641056150845207)","(99.0, -4.729192401591447)","(99.0, -4.188086880757359)","(99.0, -5.7076091634094)","(99.0, -1.2447970907362969)","(99.0, -1.1637740155191585)","(99.0, 2.6085501016671357)","(99.0, -5.29782969666985)","(2.82, -1.3678327030067237)","(99.0, -3.1692641838765283)","(99.0, 0.2662856779320147)","(99.0, -4.2803591380013915)","(99.0, -3.414721545978889)","(99.0, -2.56567861206834)","(-5.63, -1.762674327927785)","(99.0, -2.5215958033605497)","(99.0, -5.885154384767046)","(99.0, -2.5448155232825718)"
1,"(4.08, 2.321266037376194)","(-0.29, 4.649805098364714)","(6.36, 4.516855415102025)","(4.37, 6.926911094603254)","(-2.38, 3.2225104649824363)","(-9.66, 2.6788149920487023)","(-0.73, 3.156791504636599)","(-5.34, 2.050749489710112)","(8.88, 4.314795986944404)","(9.22, 2.8024043667997134)","(6.75, 3.3408591374329495)","(8.64, 3.3166258378467273)","(4.42, 4.380891923823809)","(7.43, 3.7803967339250244)","(4.56, 3.6070902927987967)","(-0.97, 4.230311098449031)","(4.66, 4.24001042796497)","(-0.68, 2.060747250951214)","(3.3, 2.054197399699439)","(-1.21, 3.4639106785584692)","(0.87, 3.283015727634237)","(8.64, 4.726582862007507)","(8.35, 3.7697718964693987)","(9.17, 6.230211259255672)","(0.05, 5.089025836603487)","(7.57, 4.0139636880826055)","(4.71, 0.3507849422706839)","(0.87, 1.3156219123013815)","(-0.39, 0.6492595270318804)","(6.99, 5.304214806964104)","(6.5, 2.315354626091073)","(-0.92, -0.6482102567168444)","(7.14, 6.707402046838438)","(9.03, 4.43391363050271)","(-1.8, 0.38909565417032255)","(0.73, -0.3217916882341535)","(7.09, 5.366700627423501)","(3.4, 2.9678493783239692)","(-0.87, 4.205586551302487)","(7.91, 4.2726835015779345)",...,"(-6.7, 1.493037641699502)","(-3.35, -0.49484776193003477)","(-9.03, 3.2880406163248117)","(4.47, 5.1212872684597635)","(4.08, 2.443430799907632)","(-3.83, 0.6593242455690621)","(8.74, 5.5264113209067425)","(1.12, -0.33690463635244783)","(0.

In [0]:
#MSE
error2=(predictions2[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mse2 = sum(error2)/len(error2)
mse2

17.125268191162917